In [1]:
#import related libraries
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np

In [51]:
#import the hard-coded data from spreadsheet
df = pd.read_excel('REMIC_TEmplate New (2).xlsx', sheet_name='Pool Info')

#Define all useful variables, with 1 being 
balance_1 = df['CURRENT'][1]
WAC_1 = df['SET.'][1]/100
term_1 = df['POOL.1'][1]
balance_2 = df['CURRENT'][2]
WAC_2 = df['SET.'][2]/100
WALA = df['POOL.2'][1]
term_2 = df['POOL.1'][2]

c = df['PC.1'][1]/100

In [21]:
def payment_annuity(r,n,balance):
    #calculate the payment received as annuity
    #r: the rate applied
    #n: term
    #balance: current balance, or present value
    
    return balance*r/(1-1/((1+r)**n))

In [56]:
def Total_Cash_Flow(WAC, init_balance, age,term, period=12,PSA=1.5):
    #Calculate the payment information for each pool
    #period: number of compoundings per year
    #term: total life of the security
    #WAC: weighted average coupon 
    #init_balance: original balance of the scurity
    
    #rate applied to every period
    r = WAC/period
    
    #payment received every period
    payment = np.zeros(term)
    
    #interest amount in the payment
    interest = np.zeros(term)
    
    #principal amount in the payment
    principal = np.zeros(term)
    
    #pre-paid principal
    prepay = np.zeros(term)
    
   #the ending balance of principal after payment has been received 
    ending_balance = np.zeros(term+1)
    ending_balance[0] = init_balance
    
    for i in range(term):
        #calculate the installment amount
        payment[i] = payment_annuity(r,term-i,ending_balance[i])
        
        #calculate the interest part of the installment
        interest[i] = ending_balance[i]*r
        
        #calculate the schedule principal payment
        if ending_balance[i]-(payment[i]-interest[i])>0.0001:
            principal[i] = payment[i]-interest[i]
        else:  principal[i] = ending_balance[i]
                
        #calculate the prepayment with given PSA
        CPR = PSA*0.06*min(1,(i+1+age)/30)
        SMM = 1-(1-CPR)**(1/12)
        prepay[i] = SMM*(ending_balance[i]-principal[i])
        
        #new ending balance calculated
        ending_balance[i+1] = ending_balance[i]-principal[i]-prepay[i]

    return payment,interest,principal,prepay,ending_balance

In [90]:
#Calculate cash flow from each pool
payment_1,interest_1,principal_1,prepay_1,ending_balance_1 = Total_Cash_Flow(WAC_1, balance_1, WALA,term_1)
payment_2,interest_2,principal_2,prepay_2,ending_balance_2 = Total_Cash_Flow(WAC_2, balance_2, WALA,term_2)

#Align the 2 pools
payment_1 = np.append(payment_1,0)
interest_1 = np.append(interest_1,0)
principal_1 = np.append(principal_1,0)
prepay_1 = np.append(prepay_1,0)
ending_balance_1 = np.append(ending_balance_1,0)

#Combine to get summary cash flow 
#note that now "principal" inlcudes both scheduled and unscheduled principal
principal = principal_1 + principal_2 + prepay_1 + prepay_2

In [184]:
#initialize all bonds
#[0]: balance
#[1]:principal
#[2]: interest
#[3] == [-1]: accrued
CG  = np.zeros((2,max(term_1,term_2)+1))
VE =  np.zeros((2,max(term_1,term_2)+1))
CM = np.zeros((2,max(term_1,term_2)+1))
TC = np.zeros((2,max(term_1,term_2)+1))
CA = np.zeros((2,max(term_1,term_2)+1))
CY = np.zeros((2,max(term_1,term_2)+1))

GZ = np.zeros((4,max(term_1,term_2)+1))
CZ = np.zeros((4,max(term_1,term_2)+1))

CG[0,0] = 74800000
VE[0,0] = 5200000
CM[0,0] = 14000000
GZ[0,0] = 22000000
TC[0,0] = 20000000
CZ[0,0] = 24000000
subtotal_1 = CG[0,0] + VE[0,0] + CM[0,0] + GZ[0,0] + TC[0,0] + CZ[0,0]
CA[0,0] = 32550000
CY[0,0] = 13950000
subtotal_2 = CA[0,0] + CY[0,0]

c_monthly = c/12
for i in range(max(term_1,term_2)):
    #From the first pool
    #First,calculate the interest accuraled/paid for GZ and CZ
    GZ[2,i+1] = c_monthly*GZ[0,i]
    CZ[2,i+1] = c_monthly*CZ[0,i]
    
    #accumulate principals for CG, VE, and CM
    CG[1,i+1] = max(0,min(principal[i]*subtotal_1/(subtotal_1+subtotal_2)+CZ[2,i+1],CG[0,i]))
    CG[0,i+1] = CG[0,i] -CG[1,i+1]
    VE[1,i+1] = max(0,min(principal[i]*subtotal_1/(subtotal_1+subtotal_2)+GZ[2,i+1]+CZ[2,i+1]-CG[1,i+1],VE[0,i]))
    VE[0,i+1] = VE[0,i] -VE[1,i+1]
    CM[1,i+1] = max(0,min(principal[i]*subtotal_1/(subtotal_1+subtotal_2)+GZ[2,i+1]+CZ[2,i+1]-CG[1,i+1]-VE[1,i+1] ,CM[0,i]))
    CM[0,i+1] = CM[0,i] -CM[1,i+1]
    
    #Make appropriate accrual, and calculate its accumulated principal for GZ
    if CM[0,i+1]>0:
        GZ[-1,i+1] = GZ[2,i+1]
    else: GZ[-1,i+1] = min(CM[1,i+1],GZ[2,i+1])
    GZ[1,i+1] = max(0,min(principal[i]*subtotal_1/(subtotal_1+subtotal_2)+GZ[-1,i+1]+CZ[2,i+1]-CG[1,i+1]-VE[1,i+1]-CM[1,i+1] ,GZ[0,i]))
    GZ[0,i+1] = GZ[0,i]+GZ[-1,i+1] - GZ[1,i+1]
    
    if GZ[0,i+1] > 0:
        TC[1,i+1] = 0
    else: TC[1,i+1] = min(principal[i]*subtotal_1/(subtotal_1+subtotal_2)+CZ[2,i+1]-GZ[1,i+1],TC[0,i])
    TC[0,i+1] = TC[0,i]  - TC[1,i+1] 
    
    #Make appropriate accrual, and calculate its accumulated principal for CZ
    if TC[0,i+1]>0:
        CZ[-1,i+1] = CZ[2,i+1]
    else: CZ[-1,i+1] = min(TC[1,i+1],CZ[2,i+1])
    CZ[1,i+1] = max(0,min(principal[i]*subtotal_1/(subtotal_1+subtotal_2)+CZ[-1,i+1]-CG[1,i+1]-VE[1,i+1]-CM[1,i+1]- GZ[1,i+1] -TC[1,i+1],CZ[0,i]))
    CZ[0,i+1] = CZ[0,i]+CZ[-1,i+1] - CZ[1,i+1]
    
    #From the 2nd pool
    CA[1,i+1] = min(principal[i]*subtotal_2/(subtotal_1+subtotal_2),CA[0,i])
    CA[0,i+1]  = CA[0,i] - CA[1,i+1]
    CY[1,i+1] = min(principal[i]*subtotal_2/(subtotal_1+subtotal_2)-CA[1,i+1],CY[0,i])
    CY[0,i+1]  = CY[0,i] - CY[1,i+1]

In [198]:
balance_bonds = np.array([CG[0], VE[0], CM[0], GZ[0],TC[0],CZ[0],CA[0],CY[0]])[:,:-1]
principal_bonds = np.array([CG[1], VE[1], CM[1], GZ[1],TC[1],CZ[1],CA[1],CY[1]])[:,1:]
interest_bonds=np.array([CG[0]*c_monthly, VE[0]*c_monthly, CM[0]*c_monthly, GZ[2]-GZ[1],TC[0]*c_monthly,CZ[2]-CZ[-1],CA[0]*c_monthly,CY[0]*c_monthly])[:,:-1]
cashflow_undiscounted = principal_bonds + interest_bonds

In [199]:
cashflow_undiscounted

array([[ 951900.89994597,  990702.86831829, 1029017.60906872, ...,
              0.        ,       0.        ,       0.        ],
       [ 113333.33333333,  113333.33333333,  113333.33333333, ...,
              0.        ,       0.        ,       0.        ],
       [  58333.33333333,   58333.33333333,   58333.33333333, ...,
              0.        ,       0.        ,       0.        ],
       ...,
       [      0.        ,       0.        ,       0.        , ...,
         195361.33296135,  192411.4479954 ,  118815.70558605],
       [ 292630.5740468 ,  303907.396105  ,  315042.6176356 , ...,
              0.        ,       0.        ,       0.        ],
       [  58125.        ,   58125.        ,   58125.        , ...,
          56541.2102729 ,   55686.51312595,   34300.34062657]])